In [1]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd

# 環境変数の取得
load_dotenv()

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [2]:
# 1. Excelファイルを読み込む
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='売上データ')
# データフレームを表示して確認
df.head()

,カテゴリー,商品コード,商品名,売上日,単価,数量,原価
0,食品,1001,りんご,2023-01-01,200,50,120
1,食品,1002,バナナ,2023-01-01,150,100,80
2,食品,1003,牛乳,2023-01-02,180,80,100
3,衣服,2001,Tシャツ,2023-01-02,1500,20,800
4,衣服,2002,ジーンズ,2023-01-03,5000,10,2500


In [3]:
# 2. データをLLM用にテキスト形式に変換
# データフレーム全体を文字列に変換
sales_data_text = df.astype(str)
prompt_text = f"売上データ:\n{sales_data_text}\nこの売上データの傾向を分析してください。"
# 表示して確認
print(prompt_text)

売上データ:
    カテゴリー 商品コード      商品名         売上日    単価   数量    原価
0      食品  1001      りんご  2023-01-01   200   50   120
1      食品  1002      バナナ  2023-01-01   150  100    80
2      食品  1003       牛乳  2023-01-02   180   80   100
3      衣服  2001     Tシャツ  2023-01-02  1500   20   800
4      衣服  2002     ジーンズ  2023-01-03  5000   10  2500
..    ...   ...      ...         ...   ...  ...   ...
235    衣服  2077   レインパンツ  2023-04-28  2000   18  1000
236    食品  1085      ザクロ  2023-04-29   600   40   300
237   日用品  3077    バスブラシ  2023-04-29   400   60   200
238    衣服  2078  レインシューズ  2023-04-30  2500   15  1250
239    食品  1086    ココナッツ  2023-04-30   300   80   150

[240 rows x 7 columns]
この売上データの傾向を分析してください。


In [4]:
# 3. OpenAI APIの呼び出し

# 役割を設定
role = "あなたはマーケティング分野に精通したデータサイエンティストです。企業の成長をサポートするために、効果的なインサイトを提供します。"

# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": role},
        {"role": "user", "content": prompt_text},
    ],
)

# LLMからの回答を表示
print(response.choices[0].message.content.strip())

売上データの分析を行う際に重要なポイントは、カテゴリーごとの売上、原価、利益率などを考慮することで、全体的な傾向や特定のカテゴリのパフォーマンスを把握することです。以下に、いくつかの主要な分析の観点を示します。

### 1. 売上のカテゴリー別分析
まず、食品、衣服、日用品の各カテゴリー別での売上を集計し、どのカテゴリが最も売上を上げているかを分析します。

- **食品**
- **衣服**
- **日用品**

### 2. 売上の時系列分析
売上日を基にデータを集約し、月別や週別でのトレンドを示します。例えば、特定の月に売上が急増している場合、その原因を特定します。特定の日やイベントに売上が集中している可能性もあります。

### 3. 利益の計算
各商品について、利益を計算します。利益は以下の式で計算されます：
\[ \text{利益} = (\text{単価} - \text{原価}) \times \text{数量} \]

この利益をカテゴリごとに集計し、どのカテゴリが最も利益を生んでいるのかを把握します。

### 4. 商品別分析
各商品の売上、利益を集計して、トップセールス商品を特定します。これにより、どの商品が最も人気であるか、どの商品が最も収益を上げているかが分かります。

### 5. 潜在的なビジネス機会の特定
売上や利益が低い商品やカテゴリーを特定し、改善点を探ります。例えば、原価が高いが売上が伸び悩んでいる商品があれば、マーケティング戦略や価格設定の見直しが必要かもしれません。

### 6. 季節性の分析
特定の季節や月に売上が増加する商品やカテゴリーを特定し、それに基づくプロモーション施策を提案します。

#### 結論
このデータをもとに、各ポイントを分析することで、企業へ次のような示唆を提供できます。

- 収益性の高い商品やカテゴリーへの注力
- 売上が伸びる時期を利用したキャンペーンの実施
- 売上の悪い商品への対応策（値下げ、プロモーションなど）

具体的な数値をもとにして詳細な分析を行うことで、より明確なインサイトが得られます。


In [5]:
# 4. 分析結果をデータフレームに変換
result_list = response.choices[0].message.content.strip().split("\n")
df_out = pd.DataFrame(result_list, columns=['結果'])
print(df_out)

                                                   結果
0   売上データの分析を行う際に重要なポイントは、カテゴリーごとの売上、原価、利益率などを考慮する...
1                                                    
2                                  ### 1. 売上のカテゴリー別分析
3   まず、食品、衣服、日用品の各カテゴリー別での売上を集計し、どのカテゴリが最も売上を上げている...
4                                                    
5                                            - **食品**
6                                            - **衣服**
7                                           - **日用品**
8                                                    
9                                     ### 2. 売上の時系列分析
10  売上日を基にデータを集約し、月別や週別でのトレンドを示します。例えば、特定の月に売上が急増し...
11                                                   
12                                       ### 3. 利益の計算
13                   各商品について、利益を計算します。利益は以下の式で計算されます：
14  \[ \text{利益} = (\text{単価} - \text{原価}) \times ...
15                                                   
16         この利益をカテゴリごとに集計し、どのカテゴリが最も利益を生んでいるのかを把握します。
17                          

In [6]:
# 5. 結果をExcelファイルに保存
df_out.to_excel("売上データ分析結果.xlsx", index=False)

In [9]:
# ワークフロー化
print("処理を開始します。")

# 1. Excelファイルを読み込む
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='売上データ')

# 2. データをLLM用にテキスト形式に変換
sales_data_text = df.astype(str)
prompt_text = f"売上データ:\n{sales_data_text}\nこの売上データの傾向を分析してください。"

# 3. OpenAI APIの呼び出し
# 役割を設定
role = "あなたはマーケティング分野に精通したデータサイエンティストです。企業の成長をサポートするために、効果的なインサイトを提供します。"
# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": role},
        {"role": "user", "content": prompt_text},
    ],
)

# 4. 分析結果をデータフレームに変換
result_list = response.choices[0].message.content.strip().split("\n")
df_out = pd.DataFrame(result_list, columns=['結果'])

# 5. 結果をExcelファイルに保存
df_out.to_excel("売上データ分析結果.xlsx", index=False)

print("Excelファイルに分析結果を保存しました。")

処理を開始します。
Excelファイルに分析結果を保存しました。
